In [495]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [496]:
cus_df=pd.read_csv('cust_data.csv')
ride_df=pd.read_csv('Trip_data.csv')
driver_df=pd.read_csv('driver_data_raw.csv')

In [498]:
cus_df.drop('Unnamed: 0',axis=1,inplace=True)
driver_df.drop('Unnamed: 0',axis=1,inplace=True)
ride_df.drop('Unnamed: 0',axis=1,inplace=True)

In [499]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the  distance between two points on a Earth."""


    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # Radius of the Earth in kilometers 
    r = 6371

    # Calculate the distance
    distance = r * c
    return distance

In [500]:
#  DataFrame named 'ride_df' with columns 'latitude' and 'longitude'
ride_df['distance_km'] = ride_df.apply(lambda row: haversine_distance(row['Origin_latitude'], row['Origin_longitude'], row['Dest_latitude'], row['Dest_longitude']), axis=1)


In [501]:
# Convert the 'datetime_column' to datetime format
ride_df['Trip_StartTime'] = pd.to_datetime(ride_df['Trip_StartTime'])

# Create new columns for date and time (without UTC)
ride_df['Trip_StartTime'] = ride_df['Trip_StartTime'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [502]:
ride_df=ride_df.sort_values(by='Trip_StartTime')

In [503]:
ride_df=ride_df.reset_index(drop=True)

In [504]:
# Convert 'pickup_datetime' to datetime format (skip this step if already in datetime format)
ride_df['Trip_StartTime'] = pd.to_datetime(ride_df['Trip_StartTime'])

In [505]:
def remove_outliers_iqr(series,q3):
    """Removes outliers from a pandas Series using the interquartile range (IQR) method.

    Args:
        series: A pandas Series.
        q3 : 75 percentile.

    Returns:
        A pandas Series with outliers removed.
    """
    # Calculate the first and third quartiles (Q1 and Q3) of the Series
    q1 = series.quantile(0.25)
    q3 = series.quantile(q3)
    
    # Calculate the interquartile range (IQR)
    iqr = q3 - q1
    
    # Define the lower and upper bounds for outliers
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    # Remove values outside the lower and upper bounds
    series_outliers_removed = series[(series >= lower_bound) & (series <= upper_bound)]
    
    
    return series_outliers_removed


In [506]:
ride_df['fare_amount']=remove_outliers_iqr(ride_df['fare_amount'],0.90)

In [507]:
# fill nan by median
ride_df['fare_amount'].fillna(ride_df['fare_amount'].median(),inplace=True)

In [508]:
ride_df.loc[ride_df['distance_km']>2000,'distance_km']=ride_df['distance_km'].quantile(0.75)

In [509]:
ride_df.loc[ride_df.distance_km==0,['fare_amount']]=0

In [510]:
ride_df.loc[(ride_df.distance_km >80) & (ride_df.fare_amount ),['fare_amount']]=49

In [511]:
ride_df.loc[(ride_df.distance_km < 1) & (ride_df.fare_amount >30 ),['fare_amount']] = ride_df.fare_amount.quantile(0.25)

In [512]:
driver_df['Driver_Age_range']=pd.cut(driver_df.Age,bins=[25,33,40,45,60],labels=['25-33','33-40','40-45','45-60'])

In [513]:
ride_df['distance_km_range']=pd.cut(ride_df.distance_km,bins=[-1,2.0,6,15,127],labels=['0-2KM','2.1-6KM','6.1-15KM','Above 15KM'])

In [514]:
cus_df['Cust_Age_range']=pd.cut(cus_df.Age,bins=[20,30,40,45,60],labels=['Young','Adult','Elder','Old age'])

In [515]:
ride_df=ride_df.loc[~(ride_df.Origin_longitude <= -91)]

In [516]:
ride_df=ride_df.loc[ride_df['distance_km'] < 100]

In [517]:
ride_df.reset_index(drop=True,inplace=True)

In [518]:
s=ride_df.loc[ride_df.distance_km!=0].index.to_list()


In [519]:
ride_df['hours_range']=pd.cut(ride_df.hour,bins=[-1,3,6,8,11,14,17,21,24],labels=['12:00AM-03:00AM','03:00AM-06:00AM','06:00AM-09:00AM','09:00AM-12:00PM','12:00PM-03:00PM','03:00PM-06:00PM','06:00PM-09:00PM','12:00AM-3:00AM'])
# pd.cut(driver_df.Age,bins=[26,33,40,45,60],labels=['Young','Adult','Elder','Old age'])

In [520]:
ride_df['fare_amount_range']=pd.cut(ride_df.fare_amount,bins=[-1,5,20,40,500],labels=['Inexpensive','Affordable','Economical','Expensive'])

In [521]:
s2=ride_df.groupby(['year','month'])['Trip_ID'].count().values

### ride

In [522]:
GROUP_DF=pd.merge(cus_df,ride_df,how='inner',on='Customer_ID').groupby('Customer_ID')

In [523]:
SPENDING_DATA=pd.DataFrame()

In [524]:
s1=GROUP_DF['fare_amount'].sum()/GROUP_DF['fare_amount'].count()

In [525]:
SPENDING_DATA['Customer_ID']=s1.index.values

In [526]:
SPENDING_DATA['SPENDING_RATIO']=s1.values

In [527]:
s2=GROUP_DF['fare_amount'].count()

In [528]:
SPENDING_DATA['SPENDING_GROUP']=pd.cut(SPENDING_DATA['SPENDING_RATIO'],bins=[-1,SPENDING_DATA.SPENDING_RATIO.quantile(0.18),SPENDING_DATA.SPENDING_RATIO.quantile(0.85),SPENDING_DATA.SPENDING_RATIO.quantile(0.95),SPENDING_DATA.SPENDING_RATIO.max()],labels=['Low Spender ','Medium Spender','High Spender','Very High Spender'])

In [529]:
driver_df['REVENUE_GENERATED_BY_DRIVER']=pd.merge(ride_df,driver_df,how='inner',on='Driver_ID').groupby('Driver_ID')['fare_amount'].sum().values

In [530]:
cus_df=pd.merge(cus_df,SPENDING_DATA,how='outer')

In [531]:
cus_df.SPENDING_RATIO.fillna(0,inplace=True)

In [532]:
driver_df['REVENUE_GENERATED_BY_DRIVER_GROUP']=pd.cut(driver_df.REVENUE_GENERATED_BY_DRIVER,bins=[-1,driver_df.REVENUE_GENERATED_BY_DRIVER.quantile(0.18),driver_df.REVENUE_GENERATED_BY_DRIVER.quantile(0.85),driver_df.REVENUE_GENERATED_BY_DRIVER.quantile(0.95),driver_df.REVENUE_GENERATED_BY_DRIVER.max()],labels=['LOW REVENUE','MEDIUM REVENUE','HIGH REVENUE','VERY HIGH REVENUE'])

In [591]:
driver_df.REVENUE_GENERATED_BY_DRIVER.max()

5173.46

In [533]:
ride_df=ride_df.loc[~(ride_df.fare_amount < 0)] # error 

In [534]:
ride_df.reset_index(drop=True,inplace=True)

In [535]:
ride_df['CALENDAR_DATE']=ride_df['date'].astype('string') + '/'+ ride_df.month.astype('string') + '/'+ ride_df.year.astype('string')

In [536]:
ride_df.loc[ride_df.Region=='Pubs',['Region']]='Restaurants'

In [537]:
ride_df.loc[ride_df.Region=='0.0',['Region']]='Market'

In [538]:
ride_df.reset_index(inplace=True,drop=True)

In [539]:
# ride_df.to_csv('trip_data_final_1.csv',index=False)

In [540]:
cancelled_trip_df=ride_df[ride_df.Trip_cancellation==1]
cancelled_trip_df=cancelled_trip_df[['Trip_ID','Trip_StartTime','Origin_longitude','Origin_latitude','Dest_longitude','Dest_latitude','Customer_ID','Driver_ID','driver_rated_for_trip','customer_rated_for_trip','Trip_cancellation','Cancelled_by','Reason','year','month','Region','CALENDAR_DATE','hours_range']]

In [541]:
cancelled_trip_df.reset_index(inplace=True,drop=True)

In [542]:
# cancelled_trip_df.to_csv('cancelled_data_final_1.csv',index=False)

In [543]:
# driver_df.to_csv('driver_data_final_1.csv',index=False)

In [544]:
cus_df.reset_index(drop=True,inplace=True)

In [546]:
# cus_df.to_csv('customer_data_final_1.csv',index=False)

In [559]:
# ride_df.to_csv('trip_data_final_2.csv',index=False)

In [563]:
# cus_df.to_csv('customer_data_final_2.csv',index=False)

In [570]:
driver_df

,Name,Age,Driver_ID,Gender,Driver_signup_date,Driver_Age_range,REVENUE_GENERATED_BY_DRIVER,REVENUE_GENERATED_BY_DRIVER_GROUP
0,John,30,D66365,Male,2014-10-17,25-33,4620.27,MEDIUM REVENUE
1,Michael,40,D13828,Male,2014-10-15,33-40,4519.12,MEDIUM REVENUE
2,Robert,52,D98272,Male,2014-10-15,45-60,4331.77,MEDIUM REVENUE
3,James,35,D63151,Male,2014-10-15,33-40,4480.55,MEDIUM REVENUE
4,William,45,D22724,Male,2014-10-15,40-45,4047.63,LOW REVENUE
5,David,28,D78629,Male,2014-10-15,25-33,4641.14,MEDIUM REVENUE
6,Richard,32,D30700,Male,2014-10-15,25-33,4775.84,MEDIUM REVENUE
7,Charles,50,D10560,Male,2014-10-15,45-60,4328.11,MEDIUM REVENUE
8,Joseph,42,D36024,Male,2014-10-15,40-45,4969.27,HIGH REVENUE
9,Thomas,38,D49646,Male,2014-10-15,33-40,4577.99,MEDIUM REVENUE


In [593]:
ride_df.columns

Index(['Trip_ID', 'fare_amount', 'Trip_StartTime', 'Origin_longitude',
       'Origin_latitude', 'Dest_longitude', 'Dest_latitude', 'passenger_count',
       'Customer_ID', 'Driver_ID', 'driver_rated_for_trip',
       'customer_rated_for_trip', 'Trip_cancellation', 'Cancelled_by',
       'Reason', 'year', 'month', 'date', 'hour', 'minutes', 'seconds',
       'Region', 'distance_km', 'distance_km_range', 'hours_range',
       'fare_amount_range', 'CALENDAR_DATE'],
      dtype='object')

In [596]:
driver_df.Driver_Age_range

0     25-33
1     33-40
2     45-60
3     33-40
4     40-45
5     25-33
6     25-33
7     45-60
8     40-45
9     33-40
10    25-33
11    45-60
12    45-60
13    25-33
14    45-60
15    33-40
16    45-60
17    25-33
18    45-60
19    25-33
20    45-60
21    40-45
22    33-40
23    45-60
24    25-33
25    45-60
26    33-40
27    40-45
28    45-60
29    33-40
30    45-60
31    40-45
32    25-33
33    45-60
34    33-40
35    45-60
36    25-33
37    40-45
38    45-60
39    25-33
40    45-60
41    25-33
42    33-40
43    45-60
44    33-40
45    25-33
46    25-33
47    25-33
48    25-33
49    33-40
Name: Driver_Age_range, dtype: category
Categories (4, object): ['25-33' < '33-40' < '40-45' < '45-60']